In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
sns.set_palette('Dark2')
sns.set_style({'axes.axisbelow': True, 'axes.edgecolor': '.15', 'axes.facecolor': 'white',
               'axes.grid': True, 'axes.labelcolor': '.15', 'axes.linewidth': 1.25, 
               'figure.facecolor': 'white', 'font.family': ['sans-serif'], 'grid.color': '.15',
               'grid.linestyle': ':', 'grid.alpha': .5, 'image.cmap': 'Greys', 
               'legend.frameon': False, 'legend.numpoints': 1, 'legend.scatterpoints': 1,
               'lines.solid_capstyle': 'round', 'axes.spines.right': False, 'axes.spines.top': False,  
               'text.color': '.15',  'xtick.top': False, 'ytick.right': False, 'xtick.color': '.15',
               'xtick.direction': 'out', 'xtick.major.size': 6, 'xtick.minor.size': 3,
               'ytick.color': '.15', 'ytick.direction': 'out', 'ytick.major.size': 6,'ytick.minor.size': 3})
sns.set_context('paper')

#http://phyletica.org/matplotlib-fonts/
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
import matplotlib.gridspec as gridspec

In [3]:
from snapanalysis.config import OUTPUT_DIRECTORY as MAIN_OUTPUT_DIRECTORY
OUTPUT_DIRECTORY = os.path.join(MAIN_OUTPUT_DIRECTORY, 'ptm-response')

if not os.path.isdir(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)
    
OUTPUT_DIRECTORY_BARPLOTS = os.path.join(OUTPUT_DIRECTORY, 'barplots')

if not os.path.isdir(OUTPUT_DIRECTORY_BARPLOTS):
    os.makedirs(OUTPUT_DIRECTORY_BARPLOTS)

In [4]:
from snapanalysis.models.ptm_response.main import OUTPUT_FILE as PTM_RESPONSE_FILE
with pd.HDFStore(PTM_RESPONSE_FILE, 'r') as store:
    
    enrichment_complexes = store['ptm_stats/joint_camera_complexes']

In [5]:
import importlib
import helpers
importlib.reload(helpers)
from helpers import *

In [6]:
import urllib

print('Informative URIs for predictors:')
print()

for predictor in PREDICTOR_ORDER:
    
    uri = predictor_sorted_uri(predictor)
    
    print(f'{predictor:>20}: {uri}')

Informative URIs for predictors:

               H2A.Z: http://ife-snap-data/proteins?pdorder=H36,H26,H37,H43,H45&showsimilar=false&noclusterproteins=true
                H3ac: http://ife-snap-data/proteins?pdorder=H05,H10,H02,H12,H16,H20,H21,H25,H22,H26,H41,H43&showsimilar=false&noclusterproteins=true
             H3K4me1: http://ife-snap-data/proteins?pdorder=H05,H03,H06,H13,H16,H30,H31,H15,H09&showsimilar=false&noclusterproteins=true
             H3K4me3: http://ife-snap-data/proteins?pdorder=H02,H13,H21,H15,H11,H14,H22,H40,H41&showsimilar=false&noclusterproteins=true
         H3K9acK14ac: http://ife-snap-data/proteins?pdorder=H15,H05,H09,H02,H11,H16,H18,H06,H19,H21,H23,H22,H24,H41,H42&showsimilar=false&noclusterproteins=true
             H3K9me2: http://ife-snap-data/proteins?pdorder=H47,H27M,H47M&showsimilar=false&noclusterproteins=true
             H3K9me3: http://ife-snap-data/proteins?pdorder=H01,H27M,H01M,H03,H04,H03M,H04M&showsimilar=false&noclusterproteins=true
             

In [7]:
color_me1 = '#BBE1BB'
color_me2 = '#9FBA7D'
color_me3 = '#6F9979'
color_ac = '#6484A2'

palette = {
     'H2A.Z': '#786D9B',
     'H3K4me1': color_me1,
     'H3K4me3': color_me3,
     'H3K9acK14ac': color_ac,
     'H3K9me2': color_me2,
     'H3K9me3': color_me3,
     'H3K27ac': color_ac,
     'H3K27me2': color_me2,
     'H3K27me3': color_me3,
     'H3ac': color_ac,
     'H4K16ac': color_ac,
     'H4K20me2': color_me2,
     'H4K20me3': color_me3,
     'H4ac': color_ac,
     'DNA Methylation': '#85B3B2',
}




In [8]:
GROUPED_ORDER = [
    ['H2A.Z', 'DNA Methylation'],
    
    ['H3K4me1', 'H3K4me3'],
    ['H3ac', 'H3K9acK14ac', 'H3K27ac'],
    
    ['H3K9me2', 'H3K9me3'],
    ['H3K27me2', 'H3K27me3'],
    
    ['H4ac', 'H4K16ac'],
    ['H4K20me2', 'H4K20me3']
]

In [9]:
limma_results = load_limma_data()

In [10]:
complexes_to_plot = sorted(enrichment_complexes.reset_index().query('significant')['Complex'].unique())

In [12]:
import re

complexes_to_plot = sorted(enrichment_complexes.reset_index().query('significant')['Complex'].unique())

max_proteins = 25

for complex_ in tqdm(complexes_to_plot):
    
    # Title of plot
    name = complex_
    complex_filename = re.sub('[^a-zA-Z0-9]+', '_', complex_).lower()
    filename = f'barplot-{complex_filename}.pdf'
    
    print(complex_)
    
    proteins = members_of(complex_)
    
    if len(proteins) > max_proteins:
        print(f'Skipping {complex_} because it has more than {max_proteins} proteins')
        continue
        
    df = limma_results.loc(axis=0)[:, proteins]
    
    if (df.groupby(level='predictor').size() > 2).sum() < 10:
        print(f'Skipping {complex_} because it does not match quality criteria')
        continue
   

    # PLOT
    fig = plt.figure(figsize=(5, 6.45), 
                     constrained_layout=True)

    n_groups = len(GROUPED_ORDER)

    spec = gridspec.GridSpec(ncols=1,
                             nrows=n_groups, 
                             figure=fig,
                             height_ratios=[len(x) for x in GROUPED_ORDER])

    ax = None
    for i, predictor_subgroup in enumerate(GROUPED_ORDER):
        ax = fig.add_subplot(spec[i, :], sharex=ax)

        medians = []
        ci = []


        subgroup_df = df.loc[predictor_subgroup]
        
        significant_predictors = set()

        for predictor in predictor_subgroup:
            
            try:
                subdf = subgroup_df.loc[predictor]
                subdf_enrichment = enrichment_complexes.loc[predictor, complex_]
            except KeyError:
                medians.append(0)
                ci.append([0, 0])
                
                continue
            
            
            mu = subdf_enrichment['empirical_median']
            ci_low = subdf_enrichment['empirical_median_ci_left']
            ci_high = subdf_enrichment['empirical_median_ci_right']
            
            if subdf_enrichment['significant']:
                significant_predictors.add(predictor)
            
            err_low = mu - ci_low
            err_high = ci_high - mu

            medians.append(mu)
            ci.append([err_low, err_high])


        for y, (predictor, mu, ci) in enumerate(zip(predictor_subgroup, medians, ci)):
            ax.barh(y, mu, 
                    xerr=np.atleast_2d(ci).T,
                    color=palette[predictor],
                    error_kw=dict(elinewidth=3, capsize=5, capthick=2),
                    edgecolor='black', linewidth=1.0)
        
     
        for significant, subsubgroup_df in subgroup_df.groupby('significant'):
        
            sns.stripplot(y='predictor', x='logFC',
                          hue='predictor',
                          hue_order=predictor_subgroup,
                          dodge=False,
                          palette=palette,
                          order=predictor_subgroup,
                          size=5,
                          alpha=.8,
                          edgecolor='black',
                          linewidth=1.0,
                          jitter=0.25,
                          marker='o' if significant else 'X',
                          ax=ax, data=subsubgroup_df.reset_index())

    #     ax.axvline(0, color='#666666')
        ax.legend_.set_visible(False)

        ax.set_ylabel('')
        
        ax.yaxis.set_tick_params(length=0, labelsize=10)
        ax.xaxis.set_tick_params(length=0)
        
        for predictor, tick in zip(predictor_subgroup, ax.get_yticklabels()):
            if predictor in significant_predictors:
                tick.set_fontweight('bold')

        if i != n_groups-1:
            sns.despine(left=True, bottom=True, offset=5, ax=ax)
            for tic in ax.xaxis.get_major_ticks():
                tic.tick1On = tic.tick2On = False
                tic.label1On = tic.label2On = False
            ax.set_xlabel('')
        else:
            sns.despine(left=True, bottom=False, offset=5, ax=ax)
            ax.set_xlabel('Change to log2 H/L ratio attributed to modification', 
                          fontsize=10)

        if i == 0:
            ax.set_title(name, fontsize=10)


    fig.savefig(os.path.join(OUTPUT_DIRECTORY_BARPLOTS, filename), 
                bbox_inches='tight')
    plt.close()
#     break


  0%|          | 0/96 [00:00<?, ?it/s]

40S Ribosomal subunit
Skipping 40S Ribosomal subunit because it has more than 25 proteins
AP-2 dimers


  2%|▏         | 2/96 [00:02<02:16,  1.45s/it]

APC/C


  3%|▎         | 3/96 [00:05<02:45,  1.78s/it]

...

 98%|█████████▊| 94/96 [02:41<00:03,  1.79s/it]

ncPRC1.6


 99%|█████████▉| 95/96 [02:43<00:01,  1.86s/it]

ncPRC1.6 (exclusive subunits)


100%|██████████| 96/96 [02:44<00:00,  1.80s/it]


In [13]:
[c for c in complexes_to_plot if 'ino80' in c.lower()]

['INO80', 'INO80 (exclusive subunits)']

In [14]:
import re

complexes_to_plot = sorted(enrichment_complexes.reset_index().query('significant')['Complex'].unique())

max_proteins = 25

for complex_ in tqdm(complexes_to_plot):
    
    # Title of plot
    name = complex_
    complex_filename = re.sub('[^a-zA-Z0-9]+', '_', complex_).lower()
    filename = f'barplot-horizontal-{complex_filename}.pdf'
    
    print(complex_)
    
    proteins = members_of(complex_)
    
    if len(proteins) > max_proteins:
        print(f'Skipping {complex_} because it has more than {max_proteins} proteins')
        continue
        
    df = limma_results.loc(axis=0)[:, proteins]
    
    if (df.groupby(level='predictor').size() > 2).sum() < 10:
        print(f'Skipping {complex_} because it does not match quality criteria')
        continue
   

    # PLOT
    fig = plt.figure(figsize=(6.25, 4.3), 
                     constrained_layout=True)

    n_groups = len(GROUPED_ORDER)

    spec = gridspec.GridSpec(ncols=n_groups,
                             nrows=1, 
                             figure=fig,
                             width_ratios=[len(x) for x in GROUPED_ORDER])

    ax = None
    for i, predictor_subgroup in enumerate(GROUPED_ORDER):
        ax = fig.add_subplot(spec[:, i], sharey=ax)

        medians = []
        ci = []


        subgroup_df = df.loc[predictor_subgroup]
        
        significant_predictors = set()

        for predictor in predictor_subgroup:
            
            try:
                subdf = subgroup_df.loc[predictor]
                subdf_enrichment = enrichment_complexes.loc[predictor, complex_]
            except KeyError:
                medians.append(0)
                ci.append([0, 0])
                
                continue
            
            
            mu = subdf_enrichment['empirical_median']
            ci_low = subdf_enrichment['empirical_median_ci_left']
            ci_high = subdf_enrichment['empirical_median_ci_right']
            
            if subdf_enrichment['significant']:
                significant_predictors.add(predictor)
            
            err_low = mu - ci_low
            err_high = ci_high - mu

            medians.append(mu)
            ci.append([err_low, err_high])


        for y, (predictor, mu, ci) in enumerate(zip(predictor_subgroup, medians, ci)):
            ax.bar(y, mu, 
                   yerr=np.atleast_2d(ci).T,
                   color=palette[predictor],
                   error_kw=dict(elinewidth=3, capsize=5, capthick=2),
                   edgecolor='black', linewidth=1.0)
        
     
        for significant, subsubgroup_df in subgroup_df.groupby('significant'):
        
            sns.stripplot(x='predictor', y='logFC',
                          hue='predictor',
                          hue_order=predictor_subgroup,
                          dodge=False,
                          palette=palette,
                          order=predictor_subgroup,
                          size=5,
                          alpha=.8,
                          edgecolor='black',
                          linewidth=1.0,
                          jitter=0.25,
                          marker='o' if significant else 'X',
                          ax=ax, data=subsubgroup_df.reset_index())

    #     ax.axvline(0, color='#666666')
        ax.legend_.set_visible(False)

        ax.set_xlabel('')
        
        ax.xaxis.set_tick_params(length=0, labelsize=10, rotation=90)
        ax.yaxis.set_tick_params(length=0)
        
        for predictor, tick in zip(predictor_subgroup, ax.get_xticklabels()):
            if predictor in significant_predictors:
                tick.set_fontweight('bold')

        if i != 0:
            sns.despine(left=True, bottom=True, offset=5, ax=ax)
            for tic in ax.yaxis.get_major_ticks():
                tic.tick1On = tic.tick2On = False
                tic.label1On = tic.label2On = False
            ax.set_ylabel('')
        else:
            sns.despine(left=False, bottom=True, offset=5, ax=ax)
            ax.set_ylabel('Change to log2 H/L ratio attributed to modification', 
                          fontsize=10)

    fig.suptitle(name, fontsize=10)


    fig.savefig(os.path.join(OUTPUT_DIRECTORY_BARPLOTS, filename), 
                bbox_inches='tight')
    plt.close()
#     break


  0%|          | 0/96 [00:00<?, ?it/s]

40S Ribosomal subunit
Skipping 40S Ribosomal subunit because it has more than 25 proteins
AP-2 dimers


  2%|▏         | 2/96 [00:02<01:40,  1.07s/it]

APC/C


  3%|▎         | 3/96 [00:06<03:15,  2.10s/it]

...

 98%|█████████▊| 94/96 [03:08<00:03,  1.77s/it]

ncPRC1.6


 99%|█████████▉| 95/96 [03:09<00:01,  1.79s/it]

ncPRC1.6 (exclusive subunits)


100%|██████████| 96/96 [03:11<00:00,  1.75s/it]


Some special plots for special complex pairs

In [47]:
import re

complex_pairs_to_plot = [('SRCAP', 'INO80 (exclusive subunits)')]

COMPLEX_PALETTES = {
    'SRCAP': '#4469A7',
    'INO80 (exclusive subunits)': '#D54E75'
}

for complex_a, complex_b in tqdm(complex_pairs_to_plot):
    
    # Title of plot
    name = f'{complex_a} vs {complex_b}'
    complex_filename_a = re.sub('[^a-zA-Z0-9]+', '_', complex_a).lower()
    complex_filename_b = re.sub('[^a-zA-Z0-9]+', '_', complex_b).lower()
    filename = f'barplot-horizontal-{complex_filename_a}vs{complex_filename_b}.pdf'
    
    print(complex_a, complex_b, filename)
    
    palette_complexes = dict(zip([complex_a, complex_b], sns.color_palette('Set1', 2)))
    if complex_a in COMPLEX_PALETTES:
        palette_complexes[complex_a] = COMPLEX_PALETTES[complex_a]
    
    if complex_b in COMPLEX_PALETTES:
        palette_complexes[complex_b] = COMPLEX_PALETTES[complex_b]
    
    
    proteins_a = members_of(complex_a)
    proteins_b = members_of(complex_b)
    
    df_a = limma_results.loc(axis=0)[:, proteins_a]
    df_b = limma_results.loc(axis=0)[:, proteins_b]
    
                                          
    if (df_a.groupby(level='predictor').size() > 2).sum() < 10:
        print(f'Skipping {complex_a} because it does not match quality criteria')
        continue
    
    if (df_b.groupby(level='predictor').size() > 2).sum() < 10:
        print(f'Skipping {complex_b} because it does not match quality criteria')
        continue

    # PLOT
    fig = plt.figure(figsize=(6.25, 5.3), 
                     constrained_layout=True)

    n_groups = len(GROUPED_ORDER)

    spec = gridspec.GridSpec(ncols=n_groups,
                             nrows=1, 
                             figure=fig,
                             width_ratios=[len(x) for x in GROUPED_ORDER])

    ax = None
    for i, predictor_subgroup in enumerate(GROUPED_ORDER):
        ax = fig.add_subplot(spec[:, i], sharey=ax)

        # - complex a -----
        
        medians_a = []
        ci_a = []
        
        subgroup_df = df_a.loc[predictor_subgroup]
        
        significant_predictors = set()
        for predictor in predictor_subgroup:
            try:
                subdf = subgroup_df.loc[predictor]
                subdf_enrichment = enrichment_complexes.loc[predictor, complex_a]
            except KeyError:
                medians.append(0)
                ci.append([0, 0])
                
                continue
            
            mu = subdf_enrichment['empirical_median']
            ci_low = subdf_enrichment['empirical_median_ci_left']
            ci_high = subdf_enrichment['empirical_median_ci_right']
            
            if subdf_enrichment['significant']:
                significant_predictors.add(predictor)
            
            err_low = mu - ci_low
            err_high = ci_high - mu

            medians_a.append(mu)
            ci_a.append([err_low, err_high])
            
        # ------ Complex b -------------------------
        
        medians_b = []
        ci_b = []
        
        
        subgroup_df = df_b.loc[predictor_subgroup]
        
        
        significant_predictors = set()
        for predictor in predictor_subgroup:
            try:
                subdf = subgroup_df.loc[predictor]
                subdf_enrichment = enrichment_complexes.loc[predictor, complex_b]
            except KeyError:
                medians.append(0)
                ci.append([0, 0])
                
                continue
            
            mu = subdf_enrichment['empirical_median']
            ci_low = subdf_enrichment['empirical_median_ci_left']
            ci_high = subdf_enrichment['empirical_median_ci_right']
            
            if subdf_enrichment['significant']:
                significant_predictors.add(predictor)
            
            err_low = mu - ci_low
            err_high = ci_high - mu

            medians_b.append(mu)
            ci_b.append([err_low, err_high])
        
        # Barplots
        bar_width = 0.35
        bar_padding = 0.05
        for y, (predictor, mu, ci) in enumerate(zip(predictor_subgroup, medians_a, ci_a)):
            ax.bar(y - (bar_width/2) - (bar_padding/2), mu, 
                   yerr=np.atleast_2d(ci).T,
                   color=palette_complexes[complex_a],
                   width=bar_width,
                   error_kw=dict(elinewidth=3, capsize=5, capthick=2),
                   edgecolor='black', linewidth=1.0)
        
        
        for y, (predictor, mu, ci) in enumerate(zip(predictor_subgroup, medians_b, ci_b)):
            ax.bar(y + (bar_width/2) + (bar_padding/2), mu, 
                   yerr=np.atleast_2d(ci).T,
                   color=palette_complexes[complex_b],
                   width=bar_width,
                   error_kw=dict(elinewidth=3, capsize=5, capthick=2),
                   edgecolor='black', linewidth=1.0)
        
        # Subgroup ab
        subgroup_a = df_a.loc[predictor_subgroup].copy()
        subgroup_a['complex'] = complex_a
        subgroup_b = df_b.loc[predictor_subgroup].copy()
        subgroup_b['complex'] = complex_b
        
        subgroup_ab = pd.concat([subgroup_a.reset_index(), 
                                 subgroup_b.reset_index()])
        
        
        for significant, subsubgroup_df in subgroup_df.groupby('significant'):
        
            sns.stripplot(x='predictor', y='logFC',
                          hue='complex',
                          hue_order=[complex_a, complex_b],
                          dodge=True,
                          palette=palette_complexes,
                          order=predictor_subgroup,
                          size=4,
                          alpha=.8,
                          edgecolor='black',
                          linewidth=1.0,
                          jitter=0.25,
                          marker='o' if significant else 'X',
                          ax=ax, data=subgroup_ab)

    #     ax.axvline(0, color='#666666')
        if ax.legend_:
            ax.legend_.set_visible(False)

        ax.set_xlabel('')
        
        ax.xaxis.set_tick_params(length=0, labelsize=10, rotation=90)
        ax.yaxis.set_tick_params(length=0)
        
        if i != 0:
            sns.despine(left=True, bottom=True, offset=5, ax=ax)
            for tic in ax.yaxis.get_major_ticks():
                tic.tick1On = tic.tick2On = False
                tic.label1On = tic.label2On = False
            ax.set_ylabel('')
        else:
            sns.despine(left=False, bottom=True, offset=5, ax=ax)
            ax.set_ylabel('Change to log2 H/L ratio attributed to modification', 
                          fontsize=10)

    fig.suptitle(name, fontsize=10)


    fig.savefig(os.path.join(OUTPUT_DIRECTORY_BARPLOTS, filename), 
                bbox_inches='tight')
    plt.close()
    break


  0%|          | 0/1 [00:00<?, ?it/s]

SRCAP INO80 (exclusive subunits) barplot-horizontal-srcapvsino80_exclusive_subunits_.pdf


In [30]:
subgroup_ab

,predictor,Gene label,logFC,CI.L,CI.R,AveExpr,t,P.Value,adj.P.Val,B,df_total,moderated_t_stdev,logFC_variance,confint_half_width,neg_log10_p,neg_log10_p_adjust,significant,significant_and_large_fc,complex
0,H4K20me3,ACTR8,-0.381052,-0.855013,0.092910,-0.248728,-1.754591,0.105179,0.416856,-5.234689,11.824618,0.217174,0.056766,0.473961,0.978069,0.380014,False,False,INO80 (exclusive subunits)
1,H4K20me3,INO80B,-0.426247,-1.035068,0.182575,0.079809,-1.527939,0.152825,0.494878,-5.264421,11.824618,0.278968,0.093666,0.608821,0.815804,0.305502,False,False,INO80 (exclusive subunits)
2,H4K20me3,ACTR5,-0.385842,-0.993680,0.221997,0.072495,-1.385338,0.191526,0.540702,-5.796292,11.824618,0.278518,0.093364,0.607839,0.717773,0.267042,False,False,INO80 (exclusive subunits)
3,H4K20me3,TFPT,-0.158337,-0.747202,0.430528,0.099785,-0.586815,0.568363,0.834038,-5.913013,11.824618,0.269824,0.087626,0.588865,0.245374,0.078814,False,False,INO80 (exclusive subunits)
4,H4K20me3,INO80D,-0.258312,-0.857101,0.340477,0.098878,-0.941469,0.365302,0.700374,-5.928753,11.824618,0.274371,0.090604,0.598789,0.437348,0.154670,False,False,INO80 (exclusive subunits)
5,H4K20me3,UCHL5,-0.191518,-0.737648,0.354611,0.031103,-0.765331,0.459078,0.765718,-5.994632,11.824618,0.250242,0.075369,0.546130,0.338113,0.115931,False,False,INO80 (exclusive subunits)
6,H4K20me3,INO80E,-0.190981,-0.862252,0.480290,0.143307,-0.620909,0.546442,0.825434,-6.030738,11.824618,0.307584,0.113867,0.671271,0.262456,0.083318,False,False,INO80 (exclusive subunits)
7,H4K20me3,NFRKB,-0.413382,-1.089447,0.262682,-0.150890,-1.334438,0.207188,0.555090,-6.069448,11.824618,0.309780,0.115499,0.676064,0.683634,0.255637,False,False,INO80 (exclusive subunits)
8,H4K20me3,INO80C,-0.267252,-0.911032,0.376529,0.101501,-0.905978,0.383037,0.720934,-6.129865,11.824618,0.294987,0.104732,0.643780,0.416759,0.142104,False,False,INO80 (exclusive subunits)
9,H4K20me3,INO80,-0.313751,-0.928376,0.300874,0.089740,-1.114062,0.287385,0.643602,-6.284974,11.824618,0.281628,0.095460,0.614625,0.541535,0.191383,False,False,INO80 (exclusive subunits)
